In [15]:
import numpy
from sklearn.feature_extraction import DictVectorizer 
import h2o 
from h2o.estimators.kmeans import H2OKMeansEstimator 
import pandas 
from function import * 
pandas.options.display.max_rows=1000 

In [16]:
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,1 hour 9 mins
H2O cluster version:,3.10.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_I_20545h
H2O cluster total nodes:,1
H2O cluster free memory:,3.186 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


#Rooms Level 

In [17]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [18]:
features_room_level, y_room_level = rooms_records(records) 

In [19]:
vectorizer_room_level = DictVectorizer() 
X_room_level = vectorizer_room_level.fit_transform(features_room_level)

In [20]:
room_level = pandas.DataFrame(X_room_level.toarray(), columns=normalize_features(vectorizer_room_level.get_feature_names())) 
room_level['type'] = y_room_level 

In [21]:
room_level_water = h2o.H2OFrame.from_python(room_level.to_dict('list')) 

In [22]:
columns = room_level_water.columns.copy() 

In [29]:
kmeans_room_level = H2OKMeansEstimator(k=2)

In [30]:
xc = columns.copy()
xc.remove('_Size') 

In [31]:
kmeans_room_level.train(training_frame=room_level_water, x=xc)

#####Standartized Centroids 

In [32]:
centroids_room_level_std = pandas.DataFrame(kmeans_room_level.centers_std(), index = xc.copy()) 

In [33]:
if centroids_room_level_std.loc['type'][0] == 'apartments': 
    centroids_room_level_std.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_room_level_std.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_room_level_std.drop('type', inplace=True)
centroids_room_level_std['Difference'] = centroids_room_level_std['Apartments'] - centroids_room_level_std['Hotels']
centroids_room_level_std.sort_values('Difference', ascending=False)

,Hotels,Apartments,Difference
_Stovetop,-0.330445,1.64173,1.97217
_Kitchenware,-0.320076,1.59021,1.91028
_Oven,-0.313384,1.55696,1.87034
_Kitchen,-0.292236,1.45189,1.74413
_Dining_area,-0.291536,1.44842,1.73995
_Toaster,-0.274717,1.36485,1.63957
_Dining_table,-0.257604,1.27983,1.53744
_Washing_Machine,-0.236562,1.17529,1.41186
_Sofa,-0.234835,1.16671,1.40154
_Electric_kettle,-0.225327,1.11947,1.3448


#####Centroids 

In [34]:
centroids_room_level = pandas.DataFrame(list(zip(*kmeans_room_level.centers())), index=xc.copy()) 
if centroids_room_level.loc['type'][0] == 'apartments': 
    centroids_room_level.rename(columns={0: 'Apartments', 1: 'Hotels'}, inplace=True)
else: 
    centroids_room_level.rename(columns={0: 'Hotels', 1: 'Apartments'}, inplace=True)
centroids_room_level.drop('type', inplace=True)
centroids_room_level['Difference'] = centroids_room_level['Apartments'] - centroids_room_level['Hotels']
centroids_room_level.sort_values('Difference', ascending=False) 

,Hotels,Apartments,Difference
_Kitchenware,0.0829563,0.866807,0.783851
_Stovetop,0.0436808,0.779077,0.735397
_Dining_area,0.0683423,0.737475,0.669132
_Kitchen,0.0542159,0.696603,0.642387
_Oven,0.00858073,0.580521,0.571941
_Sofa,0.113941,0.684312,0.570371
_Toaster,0.0358956,0.582806,0.54691
_Refrigerator,0.429496,0.968267,0.53877
_Electric_kettle,0.090815,0.603824,0.513009
_Microwave,0.282387,0.769711,0.487324
